hey

In [1]:
import pygame
import moderngl
import numpy
import glm
from LoadObject import getObjectData

pygame 2.5.2 (SDL 2.28.3, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()

WIDTH = 840
HEIGHT = 480

clock = pygame.time.Clock()
running = True

pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION, 3)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION, 3)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_FORWARD_COMPATIBLE_FLAG, True)

In [3]:
pygame.display.set_mode((WIDTH, HEIGHT), flags=pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title="Assignment 05: Mustaeen Ahmed")
gl = moderngl.get_context()
gl.info['GL_VERSION']

'4.1 Metal - 89.4'

In [4]:
geomData, bound = getObjectData("teapot_with_texCoords.obj", texture=True)
geomBuffer = gl.buffer(geomData)

Vertex position Count:  530 (5.929688, 4.125, 0.0)
Vertex normal Count:  530 (-0.966742, -0.255752, 0.0)
Vertex texture coordinate Count:  800 (2.0, 2.0, 0.0)
Face count:  992.0
texture exists


In [5]:
face_vertex_shader_code = '''
    #version 330 core

    layout (location=0) in vec3 position;
    layout (location=1) in vec2 uv;

    out vec2 f_uv;
    uniform mat4 view, perspective;

    void main() {
        f_uv = uv;
        gl_Position = perspective * view * vec4(position, 1.0);
    }
'''

face_fragment_shader_code = '''
    #version 330 core
    in vec2 f_uv;
    uniform sampler2D map;

    out vec4 color;

    void main() {
        color = texture(map, f_uv);
    }
'''

In [6]:
face_program = gl.program(
    vertex_shader=face_vertex_shader_code,
    fragment_shader=face_fragment_shader_code,
)

face_renderable = gl.vertex_array(
    face_program, [(geomBuffer, "3f 2f", "position", "uv")]
)

In [7]:
texture_img = pygame.image.load("brick.jpg")
texture_data = pygame.image.tobytes(texture_img, "RGB", True)
faceTexture = gl.texture(texture_img.get_size(), data=texture_data, components=3)

faceTexture.build_mipmaps()

faceSampler = gl.sampler(texture=faceTexture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR))

In [8]:
displacement = 2 * bound.radius * glm.rotate(glm.vec3(0, 1, 0), glm.radians(60), glm.vec3(1, 0, 0))
target_point = glm.vec3(bound.center)
up_vector = glm.vec3(0, 1, 0)

fov = glm.radians(45)
aspect = WIDTH / HEIGHT
near = bound.radius
far = 3 * bound.radius

perspective_matrix = glm.perspective(fov, WIDTH / HEIGHT, near, far)

AttributeError: module 'glm' has no attribute 'rotate'

In [ ]:
running = True
clock = pygame.time.Clock()
alpha = 0
gl.enable(gl.DEPTH_TEST)

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == 27):
            running = False
        elif (event.type == pygame.WIDNOWRESIZED):
            width = event.x
            height = event.y
            perspectiveMatrix = glm.perspective(fov, width / height, near, far)

eye_point = glm.vec3(bound.center) + glm.rotate(displacement, glm.radians(alpha), glm.vec3(0, 1, 0))
viewMatrix = glm.lookAt(eye_point, target_point, up_vector)

gl.clear(0.0, 0.0, 0.0)


face_program['view'].write(viewMatrix)
face_program['perspective'].write(perspectiveMatrix)

faceSampler.use(0)
face_program['map'] = 0
face_renderable.render(moderngl.TRIANGLES)

pygame.display.flip()
clock.tick(60)

alpha += 1
if alpha > 360:
    alpha = 0

pygame.display.quit()